# QNSPSA vs SPSA

## Packages

In [12]:
import pennylane as qml 
from pennylane import numpy as np
from sklearn import datasets 

## Data Generation 

Data generation using `sklearn` with `1000` samples and `52`features like the German dataset. 

In [13]:
data = datasets.make_classification(n_samples=1000, n_features=52, n_classes=2) 

## SPSA

In [14]:
num_qubits = 8 #8
dev = qml.device('lightning.qubit', wires = num_qubits, shots=10000) # lightning.qubit


### Functions 

In [22]:
def amplitudes(f=None, num_qubits=None):
    qml.AmplitudeEmbedding(features=f, pad_with=0.,wires=range(num_qubits),normalize=True)

@qml.qnode(dev, interface="autograd") # , interface="autograd"
def circuit(weights, x, num_qubits, depth):
    '''
    Parametrized circuit with data encoding (statepreparation) and layer repetition based on the weights 
    Args:
        weights: angles for the rotations (num layer, num qubits, num directions)
        x: input vector
    Return: 
        Expectation values measured on Pauli Z operators for the state 0
    '''
    # data encoding 
    amplitudes(x, num_qubits=num_qubits)

    # ansatz 
    #for W in weights:
    ansatz_2(weights,num_qubits=num_qubits, depth=depth)

    # measure
    return qml.expval(qml.PauliZ(0))

def variational_classifier(weights, x, num_qubits, depth, bias):
    '''
    Build the parametrized circuit with weights, x and bias term
    Args:
        - weights: rotation angles 
        - bias: classical term to add more freedom to the VQA
        - x: input vector/data 
    Returns: 
        - parametrized circuit with a bias term 
    '''
    return circuit(weights, x, num_qubits, depth) + bias

def square_loss(labels, predictions):
    '''
    Compute the cost function
    Args:
        - labels: Ground truth
        - predictions: Predicted values 
    Returns: 
        - Mean of the square error between labels and predictions = model's error 
    '''
    
    # We use a call to qml.math.stack to allow subtracting the arrays directly
    #print(labels, predictions)
    return np.mean((labels - qml.math.stack(predictions)) ** 2)


def accuracy(labels, predictions):
    '''
    Compute the accuracy of the model
    Args:
        - labels: Ground truth
        - predictions: Predicted values 
    Returns: 
        - accuracy
    '''
    acc = sum(abs(l - p) < 1e-5 for l, p in zip(labels, predictions))
    acc = acc / len(labels)
    return acc

def cost(weights, num_qubits, depth, bias, X, Y):
    '''
    Compute the cost of the model
    Args: 
        - weights: rotation angles 
        - bias: classical term to add more freedom to the VQA
        - X: input vector/data 
        - Y: True labels 
    Returns: 
        - Error prediction / distance 
    '''
    
    predictions = [variational_classifier(weights, x, num_qubits, depth, bias)._value.tolist() for x in X]
    #print(predictions)
    return square_loss(Y, predictions)

def cost2(weights, num_qubits, depth, bias, X, Y):
    '''
    Compute the cost of the model
    Args: 
        - weights: rotation angles 
        - bias: classical term to add more freedom to the VQA
        - X: input vector/data 
        - Y: True labels 
    Returns: 
        - Error prediction / distance 
    '''
    
    predictions = [variational_classifier(weights, x, num_qubits, depth, bias) for x in X]
    #print(predictions)
    return square_loss(Y, predictions)

@qml.qnode(dev)
def ansatz_2(theta:list, num_qubits=10, depth=1):
    '''
    
    '''
    step = 0
    for _ in range(depth):
        for i in range(num_qubits):
            qml.RY(theta[i+step], wires=i)
        for i in range(num_qubits-1):
            qml.CNOT([i,i+1])
        for i in range(num_qubits):
            qml.RY(theta[i+step], wires=i)
        step += num_qubits
        
    return qml.expval(qml.PauliZ(0))

In [16]:
weights_init = 0.01 * np.random.randn(4*num_qubits, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

In [ ]:
opt = qml.SPSAOptimizer(250)
#opt = qml.NesterovMomentumOptimizer(0.5)
weights = weights_init
bias = bias_init
X_ = data[0]#np.concatenate([X_new, X_transform.to_numpy()], axis=1) #data_res.to_numpy() X_new
Y_ = data[1] #y
Y_ = Y_ * 2 - 1
depth = 2
batch_size = 5*depth #5
num_qubits = 8 #8
cost_saved = []
_cost_ref_ = 10 
for it in range(250):

    # Update the weights by one optimizer step
    batch_index = np.random.randint(0, len(X_), (batch_size,))
    X_batch = X_[batch_index]
    Y_batch = Y_[batch_index]
    
    weights, _, _, bias,_,_ = opt.step(cost2, weights, num_qubits, 2, bias, X_batch, Y_batch)
    #weights, _, _, bias,_,_ = opt.step_and_cost(circuit, X_batch, weights, num_qubits, 1)
    
    #params, loss = opt.step(cost2, weights, num_qubits, 1, bias, X_batch, Y_batch)
    #print(np.sign(variational_classifier(weights, X_batch[0], num_qubits, depth, bias)))
    # Compute accuracy
    predictions = [np.sign(variational_classifier(weights, x, num_qubits, 2, bias)) for x in X_]
    #print(list(zip(Y_ , predictions)))
    acc = accuracy(Y_, predictions)
    _cost_ = cost2(weights, num_qubits, 2, bias, X_, Y_)
    cost_saved.append(_cost_)
    if _cost_ref_ > _cost_:
        print(
            "Iter: {:5d} | Cost: {:0.7f} | Accuracy: {:0.7f} ".format(
                it + 1, cost_saved[-1], acc
            )
        )
        _cost_ref_ = _cost_

Iter:     1 | Cost: 1.8052472 | Accuracy: 0.5010000 
Iter:     2 | Cost: 1.6553679 | Accuracy: 0.5010000 
Iter:     4 | Cost: 1.6497406 | Accuracy: 0.5010000 
Iter:     5 | Cost: 1.5357549 | Accuracy: 0.5010000 
Iter:     6 | Cost: 1.4926960 | Accuracy: 0.5010000 
Iter:     7 | Cost: 1.4625214 | Accuracy: 0.5010000 
Iter:     8 | Cost: 1.3857168 | Accuracy: 0.5010000 
Iter:    12 | Cost: 1.3751447 | Accuracy: 0.5010000 
Iter:    13 | Cost: 1.3546881 | Accuracy: 0.5010000 
Iter:    14 | Cost: 1.2847192 | Accuracy: 0.5010000 
Iter:    15 | Cost: 1.2576655 | Accuracy: 0.5010000 
Iter:    16 | Cost: 1.2058155 | Accuracy: 0.5010000 
Iter:    17 | Cost: 1.2012717 | Accuracy: 0.5010000 
Iter:    18 | Cost: 1.1908020 | Accuracy: 0.5010000 
Iter:    19 | Cost: 1.1639871 | Accuracy: 0.5010000 
Iter:    20 | Cost: 1.1411140 | Accuracy: 0.5010000 
Iter:    21 | Cost: 1.1040472 | Accuracy: 0.5010000 
Iter:    23 | Cost: 1.0821143 | Accuracy: 0.5010000 
Iter:    29 | Cost: 1.0719000 | Accuracy: 0.50

# QNSPSA 

Now, the `SPSA` is working. We can test with `QNSPSA` which is a different optimizer. 

In [7]:
opt = qml.QNSPSAOptimizer(stepsize=5e-2)
dev = qml.device("lightning.qubit", wires=num_qubits)

In [9]:
@qml.qnode(dev)#, interface="autograd") # , interface="autograd"
def circuit(parameters):
    '''
    Parametrized circuit with data encoding (statepreparation) and layer repetition based on the weights 
    Args:
        weights: angles for the rotations (num layer, num qubits, num directions)
        x: input vector
    Return: 
        Expectation values measured on Pauli Z operators for the state 0
    '''
    # data encoding 
    amplitudes(parameters[1], num_qubits=parameters[2])

    # ansatz 
    #for W in weights:
    ansatz_2(parameters[0],num_qubits=parameters[2], depth=parameters[3])

    # measure
    return qml.expval(qml.PauliZ(0))

In [10]:
X_ = data[0] #data_result.to_numpy() # np.concatenate([X_new, X_transform.to_numpy()], axis=1) #data_res.to_numpy() X_new
Y_ = data[1] #y
Y_ = Y_ * 2 - 1 #2 - 3


depth = 1
batch_size = 5*depth #5


weights_init = 0.01 * np.random.randn(2*depth*num_qubits, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

weights = weights_init
bias = bias_init

num_qubits = 8 #8
cost_saved = []
for it in range(100):

    # Update the weights by one optimizer step
    batch_index = np.random.randint(0, len(X_), (batch_size,))
    X_batch = X_[batch_index]
    Y_batch = Y_[batch_index]
    
    params = (weights, X_batch, num_qubits, depth)
    params, loss = opt.step_and_cost(circuit, params)
    if i % 10 == 0:
        print(f"Step {i}: cost = {loss:.4f}")

ValueError: need at least one array to concatenate